In [1]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=1c0367a3e30fb06ae15b8451ba36cd9e3f3e732d48dc1e18deee952ed90c4444
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [6]:
spark= SparkSession.builder.appName('Housing Linear Regression').getOrCreate()

In [5]:
from google.colab import files
files.upload()

Saving housing.csv to housing.csv


{'housing.csv': b'RM,LSTAT,PTRATIO,MEDV\n6.575,4.98,15.3,504000.0\n6.421,9.14,17.8,453600.0\n7.185,4.03,17.8,728700.0\n6.998,2.94,18.7,701400.0\n7.147,5.33,18.7,760200.0\n6.43,5.21,18.7,602700.0\n6.012,12.43,15.2,480900.0\n6.172,19.15,15.2,569100.0\n5.631,29.93,15.2,346500.0\n6.004,17.1,15.2,396900.0\n6.377,20.45,15.2,315000.0\n6.009,13.27,15.2,396900.0\n5.889,15.71,15.2,455700.0\n5.949,8.26,21.0,428400.0\n6.096,10.26,21.0,382200.0\n5.834,8.47,21.0,417900.0\n5.935,6.58,21.0,485100.0\n5.99,14.67,21.0,367500.0\n5.456,11.69,21.0,424200.0\n5.727,11.28,21.0,382200.0\n5.57,21.02,21.0,285600.0\n5.965,13.83,21.0,411600.0\n6.142,18.72,21.0,319200.0\n5.813,19.88,21.0,304500.0\n5.924,16.3,21.0,327600.0\n5.599,16.51,21.0,291900.0\n5.813,14.81,21.0,348600.0\n6.047,17.28,21.0,310800.0\n6.495,12.8,21.0,386400.0\n6.674,11.98,21.0,441000.0\n5.713,22.6,21.0,266700.0\n6.072,13.04,21.0,304500.0\n5.95,27.71,21.0,277200.0\n5.701,18.35,21.0,275100.0\n6.096,20.34,21.0,283500.0\n5.933,9.68,19.2,396900.0\n5.841

In [8]:
df= spark.read.csv('housing.csv',header=True,inferSchema=True)
df.show()

+-----+-----+-------+--------+
|   RM|LSTAT|PTRATIO|    MEDV|
+-----+-----+-------+--------+
|6.575| 4.98|   15.3|504000.0|
|6.421| 9.14|   17.8|453600.0|
|7.185| 4.03|   17.8|728700.0|
|6.998| 2.94|   18.7|701400.0|
|7.147| 5.33|   18.7|760200.0|
| 6.43| 5.21|   18.7|602700.0|
|6.012|12.43|   15.2|480900.0|
|6.172|19.15|   15.2|569100.0|
|5.631|29.93|   15.2|346500.0|
|6.004| 17.1|   15.2|396900.0|
|6.377|20.45|   15.2|315000.0|
|6.009|13.27|   15.2|396900.0|
|5.889|15.71|   15.2|455700.0|
|5.949| 8.26|   21.0|428400.0|
|6.096|10.26|   21.0|382200.0|
|5.834| 8.47|   21.0|417900.0|
|5.935| 6.58|   21.0|485100.0|
| 5.99|14.67|   21.0|367500.0|
|5.456|11.69|   21.0|424200.0|
|5.727|11.28|   21.0|382200.0|
+-----+-----+-------+--------+
only showing top 20 rows



In [9]:
df.printSchema()

root
 |-- RM: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- MEDV: double (nullable = true)



In [23]:
features=df.columns[:-1]
features

['RM', 'LSTAT', 'PTRATIO']

In [24]:
target=df.columns[-1]
target

'MEDV'

In [25]:
attributes =features+[target]
attributes

['RM', 'LSTAT', 'PTRATIO', 'MEDV']

In [26]:
sample=df.select(attributes)
sample.show()

+-----+-----+-------+--------+
|   RM|LSTAT|PTRATIO|    MEDV|
+-----+-----+-------+--------+
|6.575| 4.98|   15.3|504000.0|
|6.421| 9.14|   17.8|453600.0|
|7.185| 4.03|   17.8|728700.0|
|6.998| 2.94|   18.7|701400.0|
|7.147| 5.33|   18.7|760200.0|
| 6.43| 5.21|   18.7|602700.0|
|6.012|12.43|   15.2|480900.0|
|6.172|19.15|   15.2|569100.0|
|5.631|29.93|   15.2|346500.0|
|6.004| 17.1|   15.2|396900.0|
|6.377|20.45|   15.2|315000.0|
|6.009|13.27|   15.2|396900.0|
|5.889|15.71|   15.2|455700.0|
|5.949| 8.26|   21.0|428400.0|
|6.096|10.26|   21.0|382200.0|
|5.834| 8.47|   21.0|417900.0|
|5.935| 6.58|   21.0|485100.0|
| 5.99|14.67|   21.0|367500.0|
|5.456|11.69|   21.0|424200.0|
|5.727|11.28|   21.0|382200.0|
+-----+-----+-------+--------+
only showing top 20 rows



In [28]:
from pyspark.ml.regression import LinearRegression

#Genero el modelo de regression con los parametros necesarios para evitar el sobreajuste
lr = LinearRegression(featuresCol='features', labelCol = 'MEDV', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [35]:
from pyspark.ml.feature import VectorAssembler

#ensamblo todas las columnas del modelo en un unico vector

assembler = VectorAssembler(inputCols=features,outputCol="features")



In [36]:
#agrego la columna features que contiene el assembler
df = assembler.transform(df)
df.show()

+-----+-----+-------+--------+------------------+
|   RM|LSTAT|PTRATIO|    MEDV|          features|
+-----+-----+-------+--------+------------------+
|6.575| 4.98|   15.3|504000.0| [6.575,4.98,15.3]|
|6.421| 9.14|   17.8|453600.0| [6.421,9.14,17.8]|
|7.185| 4.03|   17.8|728700.0| [7.185,4.03,17.8]|
|6.998| 2.94|   18.7|701400.0| [6.998,2.94,18.7]|
|7.147| 5.33|   18.7|760200.0| [7.147,5.33,18.7]|
| 6.43| 5.21|   18.7|602700.0|  [6.43,5.21,18.7]|
|6.012|12.43|   15.2|480900.0|[6.012,12.43,15.2]|
|6.172|19.15|   15.2|569100.0|[6.172,19.15,15.2]|
|5.631|29.93|   15.2|346500.0|[5.631,29.93,15.2]|
|6.004| 17.1|   15.2|396900.0| [6.004,17.1,15.2]|
|6.377|20.45|   15.2|315000.0|[6.377,20.45,15.2]|
|6.009|13.27|   15.2|396900.0|[6.009,13.27,15.2]|
|5.889|15.71|   15.2|455700.0|[5.889,15.71,15.2]|
|5.949| 8.26|   21.0|428400.0| [5.949,8.26,21.0]|
|6.096|10.26|   21.0|382200.0|[6.096,10.26,21.0]|
|5.834| 8.47|   21.0|417900.0| [5.834,8.47,21.0]|
|5.935| 6.58|   21.0|485100.0| [5.935,6.58,21.0]|


In [37]:
#Dividio el set train y test

(trainingData, testData) = df.randomSplit([0.8, 0.2])

In [39]:
#Entreno el modelo
model = lr.fit(trainingData)

In [40]:
#realizo las predicciones con el data test
predictions = model.transform(testData)
predictions.select("prediction", "MEDV", "features").show(5)

+------------------+--------+------------------+
|        prediction|    MEDV|          features|
+------------------+--------+------------------+
|233607.01108482524|577500.0| [3.561,7.12,20.2]|
|199352.86622387255|485100.0|[3.863,13.33,20.2]|
|343922.94731945137|338100.0|[4.973,12.64,18.4]|
| 271093.4725180762|174300.0|[5.349,19.77,20.2]|
| 211567.4909556309|497700.0|[5.412,29.55,18.6]|
+------------------+--------+------------------+
only showing top 5 rows



In [43]:
from pyspark.ml.evaluation import RegressionEvaluator

#Evalua el modelo
evaluator = RegressionEvaluator(
    labelCol="MEDV", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("r2 on test data = %g" % r2)

r2 on test data = 0.698951
